In [1]:
from optimus import Optimus

In [2]:
op = Optimus(dl=True)

/Users/faviovazquez/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from sparkdl.image import imageIO

In [4]:
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from sparkdl.image import imageIO

tulips_df = ImageSchema.readImages("flower_photos/tulips").withColumn("label", lit(1))
daisy_df = imageIO.readImagesWithCustomFn("flower_photos/daisy", decode_f=imageIO.PIL_decode).withColumn("label", lit(0))
tulips_train, tulips_test, _ = tulips_df.randomSplit([0.02, 0.02, 0.96])  # use larger training sets (e.g. [0.6, 0.4] for getting more images)
daisy_train, daisy_test, _ = daisy_df.randomSplit([0.02, 0.02, 0.96])     # use larger training sets (e.g. [0.6, 0.4] for getting more images)
train_df = tulips_train.unionAll(daisy_train)
test_df = tulips_test.unionAll(daisy_test)

# Under the hood, each of the partitions is fully loaded in memory, which may be expensive.
# This ensure that each of the paritions has a small size.
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

In [5]:
model, df_preds = op.dl.image_classifier_lr(train_df)

In [6]:
op.dl.evaluate_image_classifier(test_df, model)

0.9047619047619048

In [9]:
preds = op.dl.image_predictor("sampleimg/")

Converted 378 variables to const ops.
Converted 0 variables to const ops.


In [13]:
preds.show(truncate=False)

+-----------------------------------+
|predicted_labels                   |
+-----------------------------------+
|[picket_fence, 0.14247830212116241]|
|[daisy, 0.9532104134559631]        |
|[daisy, 0.8918145298957825]        |
+-----------------------------------+

